# SIMPLE AGENT

This simple agent is similar to the tool agent (both of them are based on their past knowledge).
The difference is that this agent doesn't have a  tool and tries to generate a cypher query to retrieve the data to answer the question.
The query is not executed, this was a first step to test how prompt can be useful too.
If you don't provide, for example, the schema of the database the generated query can be general and should be changed to be run in a real database.

In [2]:
from autogen import ConversableAgent, register_function, config_list_from_json
from tools.neo4j_tools import * # Imports all the functions
import os

In [5]:
#### LLM MODEL CONFIGURATION ####

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

# This simple prompt will give some advice to the model on how to do things.
# In this very simple version, the database schema is hard-coded.
prompt = """You are an AI helpful assistant.

### RULES ###
    - You have to explain everything you do.
    - When the question refers to something that might be contained 
      in the database you must generate the query to retrieve that data.
    - When generating a query you must use the Cypher format.

### TERMINATION ###
You can add 'TERMINATE' at the end of your message if:
    - You didn't respond with a query.
    - You think the query do not generate errors.

### DATABASE SCHEMA ### 
- nodes -
(:Movie), Describe a movie that has a title and a plot. It can also have the number of likes.
(:Person), Describe actors and directors. They have a name, a birthday and they may also have the death date.

- relationships -
(:Movie) <-[:DIRECTED]- (:Person)
(:Person) -[:ACTED-IN]-> (:Movie)
(:Person) -[:KNOWS]-> (:Person)
(:Movie) -[:type]-> (:Genre)
"""


In [ ]:
# question starts the conversation asking a simple question
question = ConversableAgent(
    "question",
    llm_config=False,
    code_execution_config=False,
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

# coder will try to translate the question from natural language to cypher 
coder = ConversableAgent(
    "coder",
    system_message = prompt,
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
    llm_config = {"config_list": config_list},
    code_execution_config=False
)


#### START OF THE CHAT ####
question.initiate_chat(
    coder,
    message = "explain with rhymes what are the differences between pocahontas and matrix",
    max_turns = 4
)